In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import sys
sys.path.insert(0,'/content/head-qa-afi/code')

In [5]:
!rm -r /content/head-qa-afi/
!git clone https://github.com/claudiaqw/head-qa-afi.git

rm: cannot remove '/content/head-qa-afi/': No such file or directory
Cloning into 'head-qa-afi'...
remote: Enumerating objects: 709, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 709 (delta 0), reused 0 (delta 0), pack-reused 707
Receiving objects: 100% (709/709), 272.41 MiB | 31.87 MiB/s, done.
Resolving deltas: 100% (422/422), done.
Checking out files: 100% (222/222), done.


In [6]:
!pip install transformers
!pip install datasets
!pip install pickle5

     |████████████████████████████████| 2.6 MB 7.6 MB/s 
     |████████████████████████████████| 3.3 MB 38.2 MB/s 
     |████████████████████████████████| 636 kB 37.9 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 9.1 MB/s 
     |████████████████████████████████| 118 kB 43.3 MB/s 
     |████████████████████████████████| 243 kB 49.2 MB/s 
     |████████████████████████████████| 132 kB 8.7 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219332 sha256=4c165e4d69ab982dbf312f9c20a7a2b8a9ce7bb6222928230d28f59f9ba429a0
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [7]:
import os
import time
import datetime
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import copy
import spacy
import pickle5 as pickle
import collections
from tqdm import tqdm_notebook, trange


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import transformers
from transformers.optimization import AdamW
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, BertModel, BertForMaskedLM
from transformers import get_linear_schedule_with_warmup

import training
from training import flat_accuracy, format_time, train_model, valid_model
from training import get_optimizer, train, train_ir, validate, validate_ir, evaluator, evaluator_ir, evaluate
from training import load_embeddings_from_file, make_embedding_matrix
from training import pad_seq, encoder_bert, encoder_bert_ir, encoder_bert_instance, encoder_bert_ir_instance
from training import evaluator_bert, evaluator_bert_ir

from ir_models import BERT_QA


%matplotlib inline
%load_ext autoreload
%autoreload 2

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/364 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

In [8]:
import pickle5 as pickle

def load_dataset_from_pickle(filename):
    with open(filename, 'rb') as handle:
        return pickle.load(handle)

def filter_by_category(dataset, category):
    filtered_dataset = []
    for instance in dataset:
        categ = instance['category']
        if categ == category:
            filtered_dataset.append(instance)
    return filtered_dataset

In [9]:
BASE_BERT = 'dccuchile/bert-base-spanish-wwm-cased'

In [10]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

Downloading:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset head_qa downloaded and prepared to /root/.cache/huggingface/datasets/head_qa/es/1.1.0/d6803d1e84273cdc4a2cf3c5102945d166555f47b299ecbc5266d582f408f8e2. Subsequent calls will reuse this data.


In [11]:
training_instances = load_dataset_from_pickle('/content/head-qa-afi/data/training_ir.pickle')
validation_instances = load_dataset_from_pickle('/content/head-qa-afi/data/validation_ir.pickle')
testing_instances = load_dataset_from_pickle('/content/head-qa-afi/data/testing_ir.pickle')

oversampled_training = load_dataset_from_pickle('/content/head-qa-afi/data/oversampled_training_ir.pickle')
undersampled_training = load_dataset_from_pickle('/content/head-qa-afi/data/undersampled_training_ir.pickle')

In [12]:
tokenizer = BertTokenizer.from_pretrained(BASE_BERT, do_lower_case=False)

In [13]:
train_inputs_0, train_masks_0, train_inputs_1, train_masks_1, train_labels = encoder_bert_ir(undersampled_training, tokenizer)
valid_inputs_0, valid_masks_0, valid_inputs_1, valid_masks_1, valid_labels = encoder_bert_ir(validation_instances, tokenizer)
test_inputs_0, test_masks_0, test_inputs_1, test_masks_1, test_labels = encoder_bert_ir(testing_instances, tokenizer)

In [14]:
train_inputs_0 = torch.tensor(train_inputs_0)
valid_inputs_0 = torch.tensor(valid_inputs_0)
test_inputs_0 = torch.tensor(test_inputs_0)

train_masks_0 = torch.tensor(train_masks_0)
valid_masks_0 = torch.tensor(valid_masks_0)
test_masks_0 = torch.tensor(test_masks_0)

train_inputs_1 = torch.tensor(train_inputs_1)
valid_inputs_1 = torch.tensor(valid_inputs_1)
test_inputs_1 = torch.tensor(test_inputs_1)

train_masks_1 = torch.tensor(train_masks_1)
valid_masks_1 = torch.tensor(valid_masks_1)
test_masks_1 = torch.tensor(test_masks_1)

train_labels = torch.tensor(train_labels)
valid_labels = torch.tensor(valid_labels)
test_labels = torch.tensor(test_labels)

In [15]:
batch_size = 64

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs_0, train_masks_0, train_inputs_1, train_masks_1, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
valid_data = TensorDataset(valid_inputs_0, valid_masks_0, valid_inputs_1, valid_masks_1, valid_labels)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

# Create the DataLoader for our test set.
test_data = TensorDataset(test_inputs_0, test_masks_0, test_inputs_1, test_masks_1, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [16]:
model = BERT_QA(pretrained_model=BASE_BERT)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bi

In [17]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 203 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (31002, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [18]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [19]:
epochs_results = train_model(model, train_dataloader, valid_dataloader, valid_model, epochs, scheduler, optimizer)


======== Epoch 1 / 3 ========
Training...


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


  Batch    40  of     84.    Elapsed: 0:11:24.
  Batch    80  of     84.    Elapsed: 0:22:50.
Running Validation...
  Accuracy: 29.88
  Validation took: 0:14:54

  Average training loss: 0.70
  Training epoch took: 0:38:37

======== Epoch 2 / 3 ========
Training...
  Batch    40  of     84.    Elapsed: 0:11:39.
  Batch    80  of     84.    Elapsed: 0:23:21.
Running Validation...
  Accuracy: 36.19
  Validation took: 0:14:56

  Average training loss: 0.68
  Training epoch took: 0:39:11

======== Epoch 3 / 3 ========
Training...
  Batch    40  of     84.    Elapsed: 0:11:32.
  Batch    80  of     84.    Elapsed: 0:23:08.
Running Validation...
  Accuracy: 31.86
  Validation took: 0:15:02

  Average training loss: 0.68
  Training epoch took: 0:39:03

Training complete!


In [32]:
model_path = f'/content/head-qa-afi/code/trained_models/bert_qa'
torch.save(model.state_dict, '/content/head-qa-afi/code/trained_models/bert_qa_new')

In [21]:
t0 = time.time()
model.eval()
acc, points = evaluate(model, validation, encoder_bert_ir_instance, evaluator_bert_ir)
print(f'DEV')
print(f'accuracy: {acc}, points: {points}')
print('----------')

acc, points = evaluate(model, testing, encoder_bert_ir_instance, evaluator_bert_ir)
print(f'TEST')
print(f'accuracy: {acc}, points: {points}')
print(format_time(time.time() - t0))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


DEV
accuracy: tensor([0.2665]), points: 90
----------
TEST
accuracy: tensor([0.2527]), points: 30
1:05:16


In [23]:
with open('/content/head-qa-afi/data/train_results_bert_qa_sig.pickle', 'wb') as handle:
    pickle.dump(epochs_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
def score_exam(model, exam, encoder, evaluator):    
    right, score = 0, 0
    for question in exam:
        acc, point = evaluator(model, question, encoder)
        right += acc
        score += point
    return right/len(exam), score

def extract_exams(dataset):
    exams = {}
    for instance in dataset:
        cuaderno = instance['name']
        if cuaderno in exams:
            exams[cuaderno].append(instance)
        else:
            exams[cuaderno] = [instance]
    return exams

def evaluate_better(model, dataloader, encoder, evaluator, pytorch_model=True, bert=False):
    if pytorch_model:
        model.eval()

    exams_dict = extract_exams(dataloader)
    accuracy, points = [], []
    for _, exam in exams_dict.items():
        acc, point = score_exam(model, exam, encoder, evaluator)
        if bert:
            accuracy.append(acc)            
        else:
            accuracy.append(acc[0])
        points.append(point)
    return np.mean(accuracy), np.mean(points), accuracy, points

In [ ]:
t0 = time.time()
model.eval()

In [29]:
acc, points, acc_list, points_list = evaluate_better(model, validation, encoder_bert_ir_instance, evaluator_bert_ir, bert=True)
acc, points, acc_list, points_list

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(0.26660207,
 15.0,
 [tensor([0.2920]),
  tensor([0.2304]),
  tensor([0.2978]),
  tensor([0.2727]),
  tensor([0.2566]),
  tensor([0.2500])],
 [38, -18, 43, 21, 6, 0])

In [30]:
acc, points, acc_list, points_list = evaluate_better(model, testing, encoder_bert_ir_instance, evaluator_bert_ir, bert=True)
acc, points, acc_list, points_list

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(0.25284943,
 2.5,
 [tensor([0.2456]),
  tensor([0.2960]),
  tensor([0.2281]),
  tensor([0.2414]),
  tensor([0.2391]),
  tensor([0.2857]),
  tensor([0.2478]),
  tensor([0.2284]),
  tensor([0.2838]),
  tensor([0.2381]),
  tensor([0.2667]),
  tensor([0.2335])],
 [-4, 41, -20, -8, -10, 33, -2, -20, 31, -11, 15, -15])

In [31]:
format_time(time.time() - t0)

'1:05:37'